# **Data Collection**

## Objectives

* Fetch data from Kaggle using a json API and prepare it for future processing

## Steps undertaken
* Import packages
* Set working directory
* Fetch data from Kaggle and prepare it for further processing.
* Clean data
* Split data into different environments

## Inputs

* Kaggle JSON file - authentication token

## Outputs

* Generate Dataset: inputs/datasets/cherryleaves_dataset/cherry-leaves 

## Additional Comments

* These steps will allow us to fecth data, clean it, divide it into the different environments in preparation for the machine learning activities.



---

# Import packages

In [4]:
import numpy
import os

## Change working directory

* Change the working directory from its current folder to its parent folder

In [6]:
current_dir = os.getcwd()
current_dir

'/workspaces/Mildew-detection/jupyter_notebooks'

**Make the parent of the current directory the new current directory**

In [7]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


**Confirm the new current directory**

In [8]:
current_dir = os.getcwd()
current_dir

'/workspaces/Mildew-detection'

# Install Kaggle

In [9]:
! pip install kaggle==1.5.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 505.0 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 1.9 MB/s eta 0:00:00ta 0:00:01
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73027 sha256=0e024baef4227b7b57cd69630d81fe5a540febe2c021db218f79593d445ca65e
  Stored in directory: /home/codeany/.cache/pip/wheels/29/da/11/144cc25aebdaeb4931b231e25fd34b394e6a5725cbb2f50106
Successfully built kaggle


**Set the kaggle configuration directory to the current working directory**

In [11]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

In [13]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherryleaves_dataset"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

 89%|█████████████████████████████████▊    | 49.0M/55.0M [00:01<00:00, 49.0MB/s]
100%|██████████████████████████████████████| 55.0M/55.0M [00:01<00:00, 47.4MB/s]


**Unzip the dowloaded file and delete the zip file**

In [15]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')

# Data Preparation

## Data Cleaning

**Check and remove non images files as they will not be used for model training**

In [17]:
def remove_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir)

    for folder in folders:
        folder_path = os.path.join(my_data_dir, folder)
        files = os.listdir(folder_path)

        i = []
        j = []
        for given_file in files:
            file_location = os.path.join(folder_path, given_file)

            if os.path.isfile(file_location):
                if not given_file.lower().endswith(image_extension):
                    os.remove(file_location)  # remove non-image file
                    i.append(1)
                else:
                    j.append(1)

        print(f"Folder: {folder_path} - has image file {len(j)}")
        print(f"Folder: {folder_path} - has non-image file {len(i)}")

In [19]:
remove_non_image_file(my_data_dir='inputs/cherryleaves_dataset/cherry-leaves')

Folder: inputs/cherryleaves_dataset/cherry-leaves/powdery_mildew - has image file 2104
Folder: inputs/cherryleaves_dataset/cherry-leaves/powdery_mildew - has non-image file 0
Folder: inputs/cherryleaves_dataset/cherry-leaves/healthy - has image file 2104
Folder: inputs/cherryleaves_dataset/cherry-leaves/healthy - has non-image file 0


---

## Split train validation test set

In [20]:
import os
import shutil
import random
import joblib


def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):

    if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
        print("train_set_ratio + validation_set_ratio + test_set_ratio should sum to 1.0")
        return

    # gets classes labels
    labels = os.listdir(my_data_dir)  # it should get only the folder name
    if 'test' in labels:
        pass
    else:
        # create train, test folders with classes labels sub-folder
        for folder in ['train', 'validation', 'test']:
            for label in labels:
                os.makedirs(name=my_data_dir + '/' + folder + '/' + label)

        for label in labels:

            files = os.listdir(my_data_dir + '/' + label)
            random.shuffle(files)

            train_set_files_qty = int(len(files) * train_set_ratio)
            validation_set_files_qty = int(len(files) * validation_set_ratio)

            count = 1
            for file_name in files:
                if count <= train_set_files_qty:
                    # move a given file to the train set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/train/' + label + '/' + file_name)

                elif count <= (train_set_files_qty + validation_set_files_qty):
                    # move a given file to the validation set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/validation/' + label + '/' + file_name)

                else:
                    # move given file to test set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/test/' + label + '/' + file_name)

                count += 1

            os.rmdir(my_data_dir + '/' + label)

The data us split following the below conventions:

* The training set is divided into a 0.70 ratio of data.
* The validation set is divided into a 0.10 ratio of data.
* The test set is divided into a 0.20 ratio of data.

In [23]:
split_train_validation_test_images(my_data_dir=f"inputs/cherryleaves_dataset/cherry-leaves",
                                   train_set_ratio=0.7,
                                   validation_set_ratio=0.1,
                                   test_set_ratio=0.2
                                   )

---

# Push files to Repo